In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
df = pd.read_csv('../data/adri_data/entrenamiento_postulaciones_vistas6.csv', encoding='utf-8')
# df.drop(['idzona', 'idciudad','idmapacalle'], axis=1, inplace=True)
df.shape

(10000174, 18)

In [4]:
mintimestamp = df['timestamp'].min()
df['timestamp'] = df['timestamp']-mintimestamp

In [5]:
mintimestamp = df['idpostulante_int'].min()
df['idpostulante_int'] = df['idpostulante_int']-mintimestamp

In [6]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000174 entries, 0 to 10000173
Data columns (total 18 columns):
idaviso                   int64
esta_online               int64
titulo                    int64
descripcion               int64
idzona                    int64
idciudad                  int64
idmapacalle               int64
idtipo_de_trabajo         int64
idnivel_laboral           int64
idnombre_area             int64
iddenominacion_empresa    int64
idpostulante              object
timestamp                 float64
valor_educacion           float64
sexo                      int64
edad                      int64
se_postulo                int64
idpostulante_int          int64
dtypes: float64(2), int64(15), object(1)
memory usage: 1.9 GB


In [ ]:
cols = df.head(0)
cols.drop(['se_postulo', 'idpostulante'], inplace=True, axis=1)
cols = cols.columns
cols
cols2 = cols.drop(['idaviso', 'timestamp', 'valor_educacion', 'idpostulante_int'])
cols2

In [7]:
# X will have the numeric data, Y will have the target class
dfix = df.loc[:,cols]

In [8]:
dfix.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000174 entries, 0 to 10000173
Data columns (total 39 columns):
idaviso                   int64
esta_online               int64
titulo_analista           int64
titulo_tecnico            int64
titulo_administrativo     int64
titulo_vendedor           int64
titulo_comercial          int64
titulo_ventas             int64
titulo_sr                 int64
titulo_importante         int64
titulo_jefe               int64
titulo_empresa            int64
titulo_asistente          int64
titulo_supervisor         int64
titulo_jr                 int64
titulo_contable           int64
desc_experiencia          int64
desc_empresa              int64
desc_trabajo              int64
desc_excluyente           int64
desc_manejo               int64
desc_importante           int64
desc_equipo               int64
desc_tareas               int64
desc_cliente              int64
desc_clientes             int64
desc_conocimientos        int64
idzona               

Index(['esta_online', 'titulo_analista', 'titulo_tecnico',
       'titulo_administrativo', 'titulo_vendedor', 'titulo_comercial',
       'titulo_ventas', 'titulo_sr', 'titulo_importante', 'titulo_jefe',
       'titulo_empresa', 'titulo_asistente', 'titulo_supervisor', 'titulo_jr',
       'titulo_contable', 'desc_experiencia', 'desc_empresa', 'desc_trabajo',
       'desc_excluyente', 'desc_manejo', 'desc_importante', 'desc_equipo',
       'desc_tareas', 'desc_cliente', 'desc_clientes', 'desc_conocimientos',
       'idzona', 'idciudad', 'idmapacalle', 'idtipo_de_trabajo',
       'idnivel_laboral', 'idnombre_area', 'iddenominacion_empresa', 'sexo',
       'edad'],
      dtype='object')

In [10]:
df[cols2] = df[cols2].astype('uint8')

In [11]:
df.head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante,timestamp,valor_educacion,sexo,edad,se_postulo,idpostulante_int
0,1112448280,1,0,0,0,1,0,0,1,0,...,5,179,54,bOj3qG4,1035900.313,-1.0,1,32,0,339741
1,1112319885,1,0,0,0,0,0,0,0,0,...,0,142,18,pz9d5Ew,7133273.612,6.0,2,26,1,411417
2,1112408821,1,0,0,0,1,0,0,0,0,...,0,182,193,BmZrEJa,3432144.612,10.0,1,21,1,134550
3,1112094756,1,0,0,0,0,0,0,0,0,...,0,25,6,EzpMQdb,8792572.612,11.0,2,27,1,163553
4,1112465432,1,0,0,0,0,0,0,1,0,...,0,15,41,96Xo6wa,488126.839,12.0,1,38,0,116653


In [12]:
X = np.array(dfix)
y = np.array(df['se_postulo']) 

In [13]:
X.shape

(10000174, 39)

In [14]:
# Despues de generar el array, ya libero memoria haciendo el join entre los resultados que tengo que predecir, y el df.
top100_full = pd.read_csv("../test_final_100k.csv", encoding='utf-8')
top100_full.head()
df = pd.merge(top100_full, df, how='inner', on=['idaviso','idpostulante'])
df.shape

(100000, 42)

In [15]:
X.shape

(10000174, 39)

In [16]:
# Split into train and test, use only 10% as test (15 flowers)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7506)
X_test

array([[1.11000748e+09, 1.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 4.00000000e+01, 5.54750000e+04],
       [1.11173851e+09, 1.00000000e+00, 0.00000000e+00, ...,
        2.00000000e+00, 2.00000000e+01, 6.08100000e+03],
       [1.11231947e+09, 1.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 2.80000000e+01, 7.01070000e+04],
       ...,
       [1.11228447e+09, 0.00000000e+00, 0.00000000e+00, ...,
        2.00000000e+00, 3.40000000e+01, 3.94918000e+05],
       [1.11247693e+09, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 3.80000000e+01, 2.45679000e+05],
       [1.11246524e+09, 1.00000000e+00, 0.00000000e+00, ...,
        2.00000000e+00, 2.40000000e+01, 1.35387000e+05]])

In [17]:
y_test

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [18]:
for i in range(1, 2):    
    knn = KNeighborsClassifier(n_neighbors=i)                 # Create KNN classifier
    knn.fit(X_train, y_train)                                 # FIT (this does nothing in normal KNN)
    pred = knn.predict(X_test)                                # predict the response
    print (str(i) + ": " + str(accuracy_score(y_test, pred))) # evaluate accuracy

# 1: 0.9169948823503965
# 2: 0.903654989733514
# 3: 0.9077424696507382
# 4: 0.9014487767237989    

1: 0.8728222891987945


# Predigo con todos los datos

In [19]:
dfix3 = df[cols]
dfix3.head()

,idaviso,esta_online,titulo_analista,titulo_tecnico,titulo_administrativo,titulo_vendedor,titulo_comercial,titulo_ventas,titulo_sr,titulo_importante,...,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,timestamp,valor_educacion,sexo,edad,idpostulante_int
0,739260,0,0,0,0,0,1,0,0,0,...,0,0,2,29,86,3.177005e+06,21.0,1,42,72459
1,739260,0,0,0,0,0,1,0,0,0,...,0,0,2,29,86,3.177005e+06,11.0,2,30,87850
2,739260,0,0,0,0,0,1,0,0,0,...,0,0,2,29,86,3.177005e+06,12.0,1,36,369423
3,758580,0,0,0,0,0,0,0,1,0,...,0,0,3,168,200,3.177005e+06,17.0,2,68,16092
4,758580,0,0,0,0,0,0,0,1,0,...,0,0,3,168,200,3.177005e+06,10.0,1,32,145452


In [20]:
# dfix3.fillna(0, inplace=True)

In [21]:
X_full_data = np.array(dfix3)
X_full_data

array([[7.39260000e+05, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 4.20000000e+01, 7.24590000e+04],
       [7.39260000e+05, 0.00000000e+00, 0.00000000e+00, ...,
        2.00000000e+00, 3.00000000e+01, 8.78500000e+04],
       [7.39260000e+05, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 3.60000000e+01, 3.69423000e+05],
       ...,
       [1.11251512e+09, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 4.00000000e+01, 2.17359000e+05],
       [1.11251512e+09, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 3.10000000e+01, 3.19904000e+05],
       [1.11251512e+09, 0.00000000e+00, 0.00000000e+00, ...,
        2.00000000e+00, 4.40000000e+01, 1.04242000e+05]])

In [22]:
X_full_data.shape

(100000, 39)

In [23]:
pred_full_data = knn.predict(X_full_data)

In [24]:
df['sepostulo'] = pred_full_data

In [25]:
df2 = df[['id', 'idaviso', 'idpostulante', 'se_postulo', 'sepostulo']]

In [26]:
# dfix2.drop(['valor_educacion', 'timestamp', 'sexo','edad','idtitulo','idzona','idciudad','idmapacalle',
#             'idtipo_de_trabajo','idnivel_laboral','idnombre_area','iddenominacion_empresa'], inplace=True, axis=1)
df2[df2['se_postulo'].isnull()].shape

(0, 5)

In [27]:
df2[df2['id'] == 594]

,id,idaviso,idpostulante,se_postulo,sepostulo
594,594,17903700,Yj5q3xX,1,0


In [28]:
# dfix2.fillna(0, inplace=True)

In [29]:
top100_full = df2[['id', 'se_postulo', 'sepostulo']]
top100_full.head()

,id,se_postulo,sepostulo
0,0,0,0
1,1,0,0
2,2,0,0
3,3,0,0
4,4,0,0


In [30]:
top100_full['se_postulo'] = top100_full['se_postulo'].astype('int')
top100_full['sepostulo'] = top100_full['sepostulo'].astype('int')
top100_full.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(100000, 3)

In [31]:
top100_full[(top100_full['sepostulo'] == 0) & (top100_full['se_postulo'] == 1)]['sepostulo'].shape

(1661,)

In [32]:
# Aca modifico lo de siempre, la linea de abajo es sólo para comprobar que quedo bien, no tiene que traer registros.
top100_full.loc[(top100_full['sepostulo'] == 0) & (top100_full['se_postulo'] == 1), 'sepostulo'] = 1
top100_full[(top100_full['sepostulo'] == 0) & (top100_full['se_postulo'] == 1)]['sepostulo']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Series([], Name: sepostulo, dtype: int32)

In [33]:
top100_full.drop('se_postulo', axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
top100_full[top100_full['sepostulo'] == 1].shape

(38174, 2)

In [35]:
top100_full.to_csv('../data/adri_data/top100k_knn.csv', encoding='utf-8', index=False)